In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='2,3,4,5'
import torch
print(torch.cuda.device_count())

0


In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='2,3,4,5'
import torch
print('CUDA_VISIBLE_DEVICES Count:',torch.cuda.device_count())


import torch
import numpy
import torch.nn as nn
import torch.nn.functional as F

# 关于word embedding， 以序列建模为例

src = ['I have a hd ,','but it is a big dog .']
tgt = ['wo you yi ge hd ,','dan shi ta shi yi zhi gou .']
src_len = 8  # for pad and position embedding 
tgt_len = 10  # for pad and position embedding 
max_pos_len = 20
# word_pad = ['P', 'S', 'E']

import numpy as np
# 从原始的句子生成字典和序号输入
def sentence2input(sentence):
    '''
    input: sentence
    output: sequence, word2idx, idx2word
    '''
    source_split_words = [s.split(' ') for s in sentence]
    word = []
    for i in [j.split(' ') for j in sentence]:
        word.extend(i)
    # generate vocabulary =================
    vocab = np.array(word)
    vocab = np.unique(vocab)
    idx2word = dict(enumerate(vocab,start=1))  # 从1开始因为pad填充0
    word2idx = {v: k for k, v in idx2word.items()}
    # word2idx = {w:i for i, w in enumerate(vocab)} 
    sequence = []
    for n in source_split_words:
        bs = [word2idx[w] for w in n]
        sequence.append(bs)
    return sequence, word2idx, idx2word


def pad(input,max_len,pad_value=0):
    '''
    input: sentence
    output: sequence, word2idx, idx2word
    '''
    import copy
    pad_ = []
    for i in input:
        # print(i)
        ii = copy.deepcopy(i)
        if len(i)<max_len:
            error_len = max_len - len(i)
            for _ in range(error_len):
                ii.append(pad_value)
        pad_.append(ii[:max_len])
    pad_ = torch.IntTensor(pad_)
    return pad_

src_input, src_vocab_word2idx, enc_vocab_idx2word = sentence2input(src)
tgt_input, tgt_vocab_word2idx, tgt_vocab_idx2word = sentence2input(tgt)
src_vocab_len = len(src_vocab_word2idx)  # 后面会用到
tgt_vocab_len = len(tgt_vocab_word2idx)  # 后面会用到
print(src_input)
print(tgt_input)
print('end!')
src_input_T = pad(src_input,src_len,pad_value=0)
tgt_input_T = pad(tgt_input,tgt_len,pad_value=0)
print(src_input_T)
print(tgt_input_T)

CUDA_VISIBLE_DEVICES Count: 0
[[3, 8, 4, 9, 1], [6, 11, 10, 4, 5, 7, 2]]
[[9, 11, 10, 4, 6, 1], [3, 7, 8, 7, 10, 12, 5, 2]]
end!
tensor([[ 3,  8,  4,  9,  1,  0,  0,  0],
        [ 6, 11, 10,  4,  5,  7,  2,  0]], dtype=torch.int32)
tensor([[ 9, 11, 10,  4,  6,  1,  0,  0,  0,  0],
        [ 3,  7,  8,  7, 10, 12,  5,  2,  0,  0]], dtype=torch.int32)


In [3]:
# 构造word embedding
model_dim = 8 # 512

src_embedding_table = nn.Embedding(src_vocab_len+1,model_dim)  # 初始化一个embedding类，shape：num_embeddings: int, embedding_dim: int
tgt_embedding_table = nn.Embedding(tgt_vocab_len+1,model_dim)  # 调用的是nn.Embedding类的forword方法，直接调用类后面一个括号就是调用该类中的forward方法
print(src_embedding_table.weight)

print(src_input_T)
src_embedding = src_embedding_table(src_input_T)
tgt_embedding = tgt_embedding_table(tgt_input_T)
print(src_embedding)

Parameter containing:
tensor([[-9.1080e-01,  7.6806e-01,  1.5772e+00,  1.0454e+00, -1.4645e+00,
          6.7724e-01, -6.8672e-01, -1.0780e+00],
        [ 5.8540e-01, -1.3271e-01, -2.1287e-01,  3.7641e-01, -3.5243e-01,
         -8.8952e-01, -2.4938e+00,  9.4424e-01],
        [ 7.1382e-01,  5.1468e-01, -1.7282e+00, -7.8384e-04, -7.4555e-02,
         -5.9510e-01,  1.6940e+00,  7.2510e-01],
        [-1.6171e-01, -1.0535e+00, -1.0095e-01, -2.2332e-01,  8.0977e-01,
         -1.4494e-01, -7.9254e-01,  7.3033e-01],
        [-1.2048e-01,  2.7927e-01,  8.3582e-01,  4.2372e-01,  5.6566e-01,
         -3.7958e-02,  2.2173e-01,  8.5504e-01],
        [ 8.9905e-03, -3.9188e-01,  1.3421e+00, -1.0208e+00,  4.8851e-02,
         -5.1760e-01,  9.3678e-01,  1.1687e+00],
        [-1.0063e+00, -4.0299e-01,  8.0176e-01, -2.5186e+00,  4.9960e-01,
          7.9723e-02,  7.5859e-01,  1.1879e+00],
        [-8.2110e-01,  3.5953e-01, -6.9386e-01, -1.7914e+00, -7.4943e-01,
         -1.1318e+00, -8.3349e-02, -7.0045e

In [4]:
# 构造position embeddings
# src_len = 8 是input长度
# 实例化nn.Embedding类
# src_pos_embedding_table = nn.Embedding(src_len+1,model_dim)  # 初始化一个embedding类，shape：num_embeddings: int, embedding_dim: int
# tgt_pos_embedding_table = nn.Embedding(tgt_len+1,model_dim)  # 调用的是nn.Embedding类的forword方法，直接调用类后面一个括号就是调用该类中的forward方法

### 构造position embeddings
$$ PE(pos, 2i) = sin(pos/10000^{2i/d_model})  $$
$$ PE(pos, 2i+1) = cos(pos/1000^{2i/d_model})  $$

 where $pos$ is the position and $i$ is the dimension. 


In [5]:
# 构造position embeddings
# src_len = 8 是input长度

# 构造全长 position embedding table
# max_pos_len = 20
pos_mat = torch.arange(max_pos_len).reshape(-1,1)  # tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19])
i_mat = torch.pow(1000, torch.arange(0,model_dim,2).reshape(1,-1)/model_dim)  # tensor([[  1.0000,   5.6234,  31.6228, 177.8279]])
pos_emb_table = torch.zeros(max_pos_len,model_dim)
pos_emb_table[:,0::2] = torch.sin(pos_mat/i_mat)
pos_emb_table[:,1::2] = torch.cos(pos_mat/i_mat)
pos_embedding = nn.Embedding(max_pos_len,model_dim)
# pos_embedding.weight  # nn.Embedding.weight随机初始化方式是标准正态分布，即均值μ=0，方差σ=1的正态分布。
pos_embedding.weight = nn.Parameter(pos_emb_table,requires_grad=False)  # 这里是修改nn.Embedding类的初始化权重.weight，改为计算出的pos_emb_table
#  torch.nn.Parameter是继承自torch.Tensor的子类，其主要作用是作为nn.Module中的可训练参数使用。它与torch.Tensor的区别就是nn.Parameter会自动被认为是module的可训练参数，即加入到parameter()这个迭代器中去；而module中非nn.Parameter()的普通tensor是不在parameter中的。

# 获取 position count
src_pos = [list(range(src_len)) for _ in src]  # 遍历样本src，src_len=8
src_pos = torch.IntTensor(src_pos)
print(src_pos)

tgt_pos = [list(range(tgt_len)) for _ in src]  # 遍历样本src，tgt_len=10
tgt_pos = torch.IntTensor(tgt_pos)
print(tgt_pos)

src_pos_embedding = pos_embedding(src_pos)  # src 和tgt 输入到一个全长的  position embedding table中
tgt_pos_embedding = pos_embedding(tgt_pos)
src_pos_embedding.size(), tgt_pos_embedding.size()

tensor([[0, 1, 2, 3, 4, 5, 6, 7],
        [0, 1, 2, 3, 4, 5, 6, 7]], dtype=torch.int32)
tensor([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]], dtype=torch.int32)


(torch.Size([2, 8, 8]), torch.Size([2, 10, 8]))

In [6]:
# 构造encode 的 self attention mask
# mask shape [batchsize, src_len,model_dim]
# mask shape [batchsize, tgt_len,model_dim]
src_real_len = [len(i.split(' ')) for i in src]
valid_encoder_pos = torch.cat([torch.unsqueeze(F.pad(torch.zeros(L,dtype=torch.int32) ,pad=[0,src_len-L],value=1),0) for L in src_real_len],0)
print(valid_encoder_pos) # 这是使用unsqueeze的方法 把1维变2维
valid_encoder_pos = torch.unsqueeze(valid_encoder_pos,2) #需要再次unsqueeze 把2维变3维，这样才能使用bmm函数，保留batch size
# print(valid_encoder_pos) 
valid_encoder_pos_metr = (torch.bmm(valid_encoder_pos, valid_encoder_pos.transpose(1,2)))  # 模拟QK.T相乘
mask_encode_self_attention = valid_encoder_pos_metr.to(torch.bool)
mask_encode_self_attention

# 下面是reshape的方法
# src_real_len = [len(i.split(' ')) for i in src]
# valid_encoder_pos = torch.cat([F.pad(torch.zeros(L,dtype=torch.int32) ,pad=[0,src_len-L],value=1) for L in src_real_len],0)
# valid_encoder_pos = valid_encoder_pos.reshape(len(src),-1)  # 这是使用reshpe的方法
# print(valid_encoder_pos)
# mask_encode_self_attention = valid_encoder_pos.to(torch.bool)
# mask_encode_self_attention


tensor([[0, 0, 0, 0, 0, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 1]], dtype=torch.int32)


tensor([[[False, False, False, False, False, False, False, False],
         [False, False, False, False, False, False, False, False],
         [False, False, False, False, False, False, False, False],
         [False, False, False, False, False, False, False, False],
         [False, False, False, False, False, False, False, False],
         [False, False, False, False, False,  True,  True,  True],
         [False, False, False, False, False,  True,  True,  True],
         [False, False, False, False, False,  True,  True,  True]],

        [[False, False, False, False, False, False, False, False],
         [False, False, False, False, False, False, False, False],
         [False, False, False, False, False, False, False, False],
         [False, False, False, False, False, False, False, False],
         [False, False, False, False, False, False, False, False],
         [False, False, False, False, False, False, False, False],
         [False, False, False, False, False, False, False, F

In [7]:
# 构造encode 的 self-ttention mask
# mask shape [batchsize, src_len,model_dim]
# mask shape [batchsize, tgt_len,model_dim]
src_real_len = [len(i.split(' ')) for i in src]
valid_encoder_pos = torch.cat([torch.unsqueeze(F.pad(torch.ones(L,dtype=torch.int32) ,pad=[0,src_len-L],value=0),0) for L in src_real_len],0)
print(valid_encoder_pos) # 这是使用unsqueeze的方法 把1维变2维
valid_encoder_pos = torch.unsqueeze(valid_encoder_pos,2) #需要再次unsqueeze 把2维变3维，这样才能使用bmm函数，保留batch size
# print(valid_encoder_pos) 


invalid_encoder_pos = 1 - valid_encoder_pos



invalid_encoder_pos_matrix = (torch.bmm(invalid_encoder_pos, invalid_encoder_pos.transpose(1,2)))  # 模拟QK.T相乘
mask_encode_self_attention = invalid_encoder_pos_matrix.to(torch.bool)
mask_encode_self_attention

# 下面是reshape的方法
# src_real_len = [len(i.split(' ')) for i in src]
# valid_encoder_pos = torch.cat([F.pad(torch.zeros(L,dtype=torch.int32) ,pad=[0,src_len-L],value=1) for L in src_real_len],0)
# valid_encoder_pos = valid_encoder_pos.reshape(len(src),-1)  # 这是使用reshpe的方法
# print(valid_encoder_pos)
# mask_encode_self_attention = valid_encoder_pos.to(torch.bool)
# mask_encode_self_attention


tensor([[1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0]], dtype=torch.int32)


tensor([[[False, False, False, False, False, False, False, False],
         [False, False, False, False, False, False, False, False],
         [False, False, False, False, False, False, False, False],
         [False, False, False, False, False, False, False, False],
         [False, False, False, False, False, False, False, False],
         [False, False, False, False, False,  True,  True,  True],
         [False, False, False, False, False,  True,  True,  True],
         [False, False, False, False, False,  True,  True,  True]],

        [[False, False, False, False, False, False, False, False],
         [False, False, False, False, False, False, False, False],
         [False, False, False, False, False, False, False, False],
         [False, False, False, False, False, False, False, False],
         [False, False, False, False, False, False, False, False],
         [False, False, False, False, False, False, False, False],
         [False, False, False, False, False, False, False, F

In [8]:
# mask 实施 测试

score = torch.randn(len(src),src_len,src_len)
score
masked_score = score.masked_fill(mask_encode_self_attention, -np.inf)
masked_score
F.softmax(masked_score,-1)

tensor([[[0.1583, 0.0269, 0.0217, 0.1735, 0.1535, 0.0742, 0.2146, 0.1774],
         [0.0581, 0.0064, 0.1319, 0.1381, 0.0196, 0.0132, 0.0164, 0.6162],
         [0.0669, 0.0729, 0.0682, 0.0754, 0.0530, 0.2847, 0.1915, 0.1876],
         [0.0220, 0.2650, 0.0197, 0.0444, 0.1443, 0.4586, 0.0097, 0.0363],
         [0.2126, 0.0731, 0.1503, 0.0604, 0.0958, 0.0727, 0.0306, 0.3045],
         [0.2245, 0.1289, 0.2679, 0.1955, 0.1831, 0.0000, 0.0000, 0.0000],
         [0.1057, 0.5387, 0.0365, 0.2916, 0.0275, 0.0000, 0.0000, 0.0000],
         [0.0085, 0.8356, 0.0229, 0.0189, 0.1141, 0.0000, 0.0000, 0.0000]],

        [[0.0599, 0.0717, 0.0885, 0.1039, 0.3555, 0.0098, 0.2548, 0.0558],
         [0.0212, 0.0192, 0.0344, 0.4755, 0.0918, 0.0125, 0.0440, 0.3014],
         [0.0125, 0.1660, 0.0209, 0.0590, 0.0802, 0.3120, 0.1218, 0.2274],
         [0.2641, 0.0281, 0.1057, 0.0245, 0.2033, 0.0554, 0.2343, 0.0847],
         [0.0961, 0.0202, 0.4509, 0.0865, 0.0368, 0.0429, 0.0759, 0.1908],
         [0.1286, 0.185

In [9]:
# 构造 intra attention mask 
src_real_len = [len(i.split(' ')) for i in src]
tgt_real_len = [len(i.split(' ')) for i in tgt]
valid_encoder_pos = torch.unsqueeze(torch.cat([torch.unsqueeze(F.pad(torch.ones(L,dtype=torch.int32) ,pad=[0,src_len-L],value=0),0) for L in src_real_len],0),2)
print(valid_encoder_pos.size())
valid_decoder_pos = torch.unsqueeze(torch.cat([torch.unsqueeze(F.pad(torch.ones(L,dtype=torch.int32) ,pad=[0,tgt_len-L],value=0),0) for L in tgt_real_len],0),2)
print(valid_decoder_pos.size())
valid_cross_pos_matrix = torch.bmm(valid_decoder_pos, valid_encoder_pos.transpose(1,2))  #注意这里 tgt-decode是Q，src-encoed是K，计算公式是Q和K转置矩阵乘
valid_cross_pos_matrix.size()
invalid_cross_pos_matrix = 1 - valid_cross_pos_matrix
mask_cross_self_attention = invalid_cross_pos_matrix.to(torch.bool)
mask_cross_self_attention

torch.Size([2, 8, 1])
torch.Size([2, 10, 1])


tensor([[[False, False, False, False, False,  True,  True,  True],
         [False, False, False, False, False,  True,  True,  True],
         [False, False, False, False, False,  True,  True,  True],
         [False, False, False, False, False,  True,  True,  True],
         [False, False, False, False, False,  True,  True,  True],
         [False, False, False, False, False,  True,  True,  True],
         [ True,  True,  True,  True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True,  True,  True,  True]],

        [[False, False, False, False, False, False, False,  True],
         [False, False, False, False, False, False, False,  True],
         [False, False, False, False, False, False, False,  True],
         [False, False, False, False, False, False, False,  True],
         [False, False, False, False, False, False, False,  

In [10]:
# 构造 decode attention mask -- 有点不一样
# 构造下三角矩阵 为什么？ 因为decoder的输入要把后面的数据mask掉 每次输入的数据向后移一位 即看到的范围越来越大
m_ = [torch.unsqueeze(F.pad(torch.tril(torch.ones((L,L))),(0,(tgt_len-L),0,(tgt_len-L))),0)  for L in tgt_real_len]
print(m_[0].size())
valide_decoder_tri_matrix = torch.cat(m_)
invalide_decoder_tri_matrix = 1 - valide_decoder_tri_matrix
invalide_decoder_tri_matrix = invalide_decoder_tri_matrix.to(torch.bool)

torch.Size([1, 10, 10])


In [11]:
invalide_decoder_tri_matrix

tensor([[[False,  True,  True,  True,  True,  True,  True,  True,  True,  True],
         [False, False,  True,  True,  True,  True,  True,  True,  True,  True],
         [False, False, False,  True,  True,  True,  True,  True,  True,  True],
         [False, False, False, False,  True,  True,  True,  True,  True,  True],
         [False, False, False, False, False,  True,  True,  True,  True,  True],
         [False, False, False, False, False, False,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True]],

        [[False,  True,  True,  True,  True,  True,  True,  True,  True,  True],
         [False, False,  True,  True,  True,  True,  True,  True,  True,  True],
         [False, False, Fa

In [12]:
# mask 实施 测试

score = torch.randn(len(src),tgt_len,tgt_len)
score
masked_score = score.masked_fill(invalide_decoder_tri_matrix, -1e9)
# print(masked_score)
F.softmax(masked_score,-1)[0]



tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.1985, 0.8015, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0932, 0.6171, 0.2897, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0899, 0.7133, 0.1502, 0.0467, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0526, 0.1542, 0.7454, 0.0334, 0.0144, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0573, 0.5793, 0.0637, 0.0693, 0.1934, 0.0370, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
         0.1000],
        [0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
         0.1000],
        [0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
         0.1000],
        [0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
         0.1000]])

In [13]:
# 构造scaled self attention 函数
def scale_dot_product_attention(Q,K,V,attention_mask):  # Q K 是 batch的， 同时，也是batch * multi-head的
    #  shape of Q K V : [batch_size,num_head,src_len,model_dim/num_head]
    score = torch.bmm(Q, K.transpose(-1,-2))/torch.sqrt(model_dim)
    masked_score = score.masked_fill(attention_mask, -1e9)
    prob = F.softmax(masked_score,-1)
    context = torch.bmm(prob,V)
    return context



In [14]:
transformer = nn.Transformer(nhead=2, num_encoder_layers=6)
src_ = torch.rand((10,32,512))
tgt_ = torch.rand((20,32,512))
out = transformer(src_,tgt_)
out.size()

torch.Size([20, 32, 512])

In [15]:
# # 4 大类
# TransformerEncoder
#     TransformerEncoderLayer

# TransformerDecoder
#     TransformerDe  coderLayer

In [16]:
# Transformer Masked loss
# 使用机器翻译任务 模拟
import torch 
import torch.nn as nn
import torch.nn.functional as F 

# 两个句子，每个句子3个单词，那就是一共6个单词
Logits = torch.randn(2,3,4) # bs=2,seqlen=3,vocab_size=class=4 
Logits = Logits.transpose(1,2)  # 在torch 官方crossentropy调用的时候需要的维度是bs，C，d 没有经过softmax之前
lable = torch.randint(0,4,(2,3))
Logits.size(),lable.size()
# 这里的交叉熵是6个单词的平均的交叉熵
F.cross_entropy(Logits,lable) 
# 返回每一个单词的交叉熵
F.cross_entropy(Logits,lable,reduction='none') 
# 构造mask len 即假设tgt的有效长度只有2，有1位是padding的，那就需要把这一位mask掉再进行loss掉统计
tgt_len = torch.Tensor([2,3]).to(torch.int32) # 假设两个样本长度为2，3

valid_mask = torch.cat([torch.unsqueeze(F.pad(torch.ones(l),(0,max(tgt_len)-l)),0) for l in tgt_len])
F.cross_entropy(Logits,lable,reduction='none') * valid_mask

# 或者
lable[0,2] = -100
F.cross_entropy(Logits,lable,reduction='none')

tensor([[3.1777, 0.5750, 0.0000],
        [1.9023, 1.1583, 1.7151]])

In [17]:
Logits,lable
# tensor([[[ 1.5035,  1.1617,  0.8024,  0.4093],
#          [ 0.0355, -0.5075, -1.2482, -0.7190],
#          [ 0.5320,  1.7259,  0.4466,  0.6156]],

# tensor([[3, 1, 1],
#          [2, 2, 3]]))

(tensor([[[ 1.8386, -1.2391,  1.2173],
          [-0.1758,  0.0315,  2.0426],
          [-1.1536,  0.6510, -0.7551],
          [-2.0580, -1.7816, -0.6529]],
 
         [[-0.1527,  0.3020, -0.4522],
          [-0.2054,  0.5440,  0.6402],
          [ 0.0865, -0.5973,  0.3185],
          [ 1.5138,  0.6209,  1.3174]]]),
 tensor([[   2,    2, -100],
         [   2,    1,    2]]))

In [18]:
lable

tensor([[   2,    2, -100],
        [   2,    1,    2]])